# Flask Application

In [ ]:
import io
import random
import matplotlib.pyplot as plt
import seaborn as sns
from flask import Response
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import os
from pathlib import Path
from flask import Flask, render_template, request, url_for, redirect
from flask_uploads import UploadSet, configure_uploads, IMAGES, DATA, ALL
from werkzeug import secure_filename
import pandas as pd 
import pygal
import io
import plotly.express as px

import os
import plotly
import plotly.graph_objs as go

import pandas as pd
import numpy as np
import json

In [ ]:
app = Flask(__name__)

files = UploadSet('files', ALL)
#destination to store file which is uploaded
app.config['UPLOADED_FILES_DEST'] = 'static/myfiles'
#allowed format
app.config['ALLOWED_EXTENSIONS'] = ['CSV']
configure_uploads(app,files)

#Only allow csv files
def allowed_extension(filename):
    if not '.' in filename:
        return False
    ext = filename.rsplit('.',1)[1]
    if ext.upper() in app.config['ALLOWED_EXTENSIONS']:
        return True 
    else:
        return False
    
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/dataupload',methods=['GET','POST'])
def dataupload():
    if request.method == 'POST' and 'csv_data' in request.files:
        file = request.files['csv_data']
        if file.filename == "":
            print('file must have a filename')
            return redirect(request.url)
        if not allowed_extension(file.filename):
            print('Extension not allowed')
            return redirect(request.url)
        else:
            filename = secure_filename(file.filename)
        
            file.save(os.path.join('static/myfiles',filename))
            fullfile = os.path.join('static/myfiles',filename)

            df = pd.read_csv(os.path.join('static/myfiles',filename))
            df_describe = df.copy()
            
            return render_template(
                'details.html', 
                filename=filename, 
                df_describe=df_describe,
            )
    else:
        return render_template("view_details.html")


@app.route('/view_details',methods=['GET','POST'])
def view_details():
    dirpath = 'static/myfiles'
    paths = sorted(Path(dirpath).iterdir(), key=os.path.getmtime)
    df = pd.read_csv(paths[-1])
    df_describe = df.copy()
    return render_template("view_details.html",df_describe=df_describe)
    
@app.route('/charts',methods=['GET','POST'])
def charts():
    
    dirpath = 'static/myfiles'
    paths = sorted(Path(dirpath).iterdir(), key=os.path.getmtime)
    df = pd.read_csv(paths[-1])
    cat = df.groupby("Category")["Sales"].sum().reset_index()
    #Category wise sales   
    data = [
        go.Bar(
            x=cat['Category'],
            y=cat['Sales']
        )
    ]

    graphJSON = json.dumps(data, cls=plotly.utils.PlotlyJSONEncoder)
    
    #State wise sales
    group_2 = df.groupby("State")["Sales"].sum().reset_index()
    country = [
            go.Bar(
                x=group_2['State'], 
                y=group_2['Sales']
            )
        ]
    graphJSON_count = json.dumps(country, cls=plotly.utils.PlotlyJSONEncoder)
            
    #Subcategory wise sales
    group_2 = df.groupby("Sub-Category")["Sales"].sum().reset_index()
    country = [
            go.Bar(
                x=group_2['Sub-Category'], 
                y=group_2['Sales']
            )
        ]
    graphJSON_sub = json.dumps(country, cls=plotly.utils.PlotlyJSONEncoder)
            
    
                  
    ############################################# Chart 4 ########## State Year wise Profit
    df["Order Date"] = pd.to_datetime(df["Order Date"])
    temp = df.pivot_table(index= "Order Date",columns="State",values="Profit").fillna(0)
    # # plotly
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=temp.index,
                             y=temp[temp.columns[0]],
                             visible=True)
                 )

    updatemenu = []
    buttons = []

    # button with one option for each dataframe
    for col in temp.columns:
        buttons.append(dict(method='restyle',
                            label=col,
                            visible=True,
                            args=[{'y':[temp[col]],
                                   'x':[temp.index],
                                   'type':'scatter'}, [0]],
                            )
                      )

    # some adjustments to the updatemenus
    updatemenu = []
    your_menu = dict()
    updatemenu.append(your_menu)

    updatemenu[0]['buttons'] = buttons
    updatemenu[0]['direction'] = 'down'
    updatemenu[0]['showactive'] = True

    # add dropdown menus to the figure
    view = fig.update_layout(showlegend=False, updatemenus=updatemenu)
        # Convert the figures to JSON
    graphJSON_new = json.dumps(view, cls=plotly.utils.PlotlyJSONEncoder)
    
    
    ############################ Chart 5
    #Subcategory wise sales
    group_5 = df.groupby("Segment")["Sales"].sum().reset_index()
    fig = px.pie(group_5, values='Sales', names='Segment')

    graphJSON_imp = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
            
    
    return render_template('chart.html',plot=graphJSON,plot_2=graphJSON_count,plot_3=graphJSON_sub,
                           filename=paths[-1],plot_4 =graphJSON_new,plot_5 = graphJSON_imp )

In [ ]:
if __name__ == '__main__':
    app.run(debug=False)